# 1.1 Load libraries

Note: After resolving conflicts, runs the entire notebook to make sure none of the old code is broken.

In [1]:
import pandas as pd
import numpy as np
import os
from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions, base_optimizer, black_litterman

# 1.3 Load the stocks into memory and perform some preprocessing steps

The data is stored as key-value pairs in a dictionary, with the ticker referencing the respective dataframe. Following that, the date is changed into a datetime object and a new column for log-returns is added.

In [2]:
# load stocks into a dictionary
stocks = {}
do_not_append = ['FSELX','IPHI', '005930'] # FSELX is our fund, while IPHI does not have data and has been bought over by MRVL. 005930 non-USD.  'ASX', 'NXPI', 'TSM' companies are not based in the US but traded in US markets
for file_name in os.listdir('best_data/'):
  ticker = file_name.split(".")[0]
  if ticker in do_not_append:
    pass
  else:
    stocks[f'{ticker}'] = pd.read_csv(f'best_data/{file_name}') # for each stock, add its ticker as the key and dataframe as the value

In [3]:
print(list(stocks.keys())) # print the stock tickers for easy referencing

['005930', 'ADI', 'AEIS', 'AMAT', 'AMBA', 'AMD', 'AOSL', 'ASX', 'AVGO', 'CRUS', 'DIOD', 'ENPH', 'FLEX', 'IIVI', 'INTC', 'JBL', 'LRCX', 'MCHP', 'MRVL', 'MTSI', 'MU', 'MXL', 'NVDA', 'NXPI', 'OLED', 'ON', 'QCOM', 'SEDG', 'SGH', 'SMTC', 'SYNA', 'TSM', 'TTMI', 'TXN', 'XLNX']


In [4]:
# Change date column into a datetime object
# Add a new column for log returns
for ticker, df in stocks.items():
  df['Date'] = df['Date'].apply(pd.to_datetime)
  df['LogReturns'] = np.append(np.nan,np.diff(np.log(df['Adj Close']))) # Fill dataframe with the log returns. The first value will be nan because there is no log returns for it.

In [5]:
# preview a stock
stocks['TXN'].head()

,Date,Open,High,Low,Close,Adj Close,Volume,LogReturns
0,2016-10-04,69.660004,69.910004,69.099998,69.589996,61.006310,4089500,NaN
1,2016-10-05,69.849998,70.839996,69.010002,70.599998,61.891724,4789600,0.014409
2,2016-10-06,70.440002,71.000000,70.029999,70.959999,62.207321,4061500,0.005086
3,2016-10-07,71.000000,71.059998,70.449997,70.870003,62.128426,3673600,-0.001269
4,2016-10-10,71.080002,71.330002,69.900002,69.930000,61.304375,4893200,-0.013352


In [6]:
top_30 = ['SYNA', 'MRVL', 'SGH', 'ENPH', 'JBL', 'CRUS', 'MCHP', 'AEIS', 'MTSI', 'AMAT', 'NVDA', 'ON', 'TTMI', 'SMTC', 'AVGO', 'XLNX', 'TSM', 'IIVI', 'QCOM', 'NXPI', 'FLEX', 'DIOD', 'AMD', 'MXL', 'ADI', 'INTC', 'TXN', 'LRCX', 'MU', 'OLED']
stocks_30 = {stock:df for stock,df in stocks.items() if stock in top_30}

# 1.4 Aggregate the returns into one dataframe

In [7]:
log_returns = pd.DataFrame(data=np.full((stocks['TXN'].shape[0],30),np.nan), columns = sorted(list(stocks_30.keys())), index = stocks_30['TXN'].Date) # create empty dataframe filled with NaNs, with index = Date and columns = stocks

# copy the log returns over from the stocks dictionary
for ticker, df in stocks_30.items():
  log_returns[ticker] = log_returns.index.map(stocks_30[ticker].set_index('Date')['LogReturns'])

# if simple returns are needed
simple_returns = np.exp(log_returns)-1
#simple_returns['RF'] = np.repeat(0.01/252,simple_returns.shape[0])

In [8]:
# preview log_returns
simple_returns.head()

,ADI,AEIS,AMAT,AMD,AVGO,CRUS,DIOD,ENPH,FLEX,IIVI,...,OLED,ON,QCOM,SGH,SMTC,SYNA,TSM,TTMI,TXN,XLNX
Date,,,,,,,,,,,,,,,,,,,,,
2016-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-05,0.006849,0.008323,0.011145,-0.027260,0.026205,0.018109,0.024402,-0.008333,0.030837,0.043100,...,-0.006262,0.018062,0.002399,NaN,0.008727,0.049058,0.004607,0.019164,0.014513,0.006596
2016-10-06,0.000928,0.027513,-0.001670,0.026549,0.004093,0.012919,0.003270,0.016807,0.004986,0.026619,...,0.003819,0.002419,0.010473,NaN,-0.002884,0.054531,0.007861,0.001709,0.005099,0.000375
2016-10-07,-0.004943,-0.015448,-0.004684,-0.030172,0.000115,-0.000739,-0.016760,-0.041322,-0.000709,-0.005418,...,-0.000761,-0.012068,0.009624,NaN,-0.004700,0.017339,0.013325,0.014505,-0.001268,-0.010107
2016-10-10,-0.016610,0.004184,-0.012437,0.013333,0.003214,0.023122,-0.007102,-0.086207,-0.004965,0.003891,...,0.013707,-0.012215,-0.013785,NaN,-0.027243,0.014630,0.003207,0.000000,-0.013264,-0.012857


The stock SGH has a lot of NaN values. We take a closer look at it.

In [9]:
# preview SGH
stocks['SGH'].head()

,Date,Open,High,Low,Close,Adj Close,Volume,LogReturns
0,2017-05-24,12.00,14.000,11.50,13.45,13.45,1415200,NaN
1,2017-05-25,13.85,14.320,13.35,13.50,13.50,237900,0.003711
2,2017-05-26,13.36,13.624,12.46,13.23,13.23,56200,-0.020203
3,2017-05-30,13.20,13.255,12.28,12.98,12.98,71200,-0.019077
4,2017-05-31,13.02,13.500,12.93,13.10,13.10,55700,0.009203


SGH only begins at a later date, explaining the NaNs we see in our earlier preview.

## 2.1 Optimization

In [10]:
original_portfolio = pd.read_csv('original_weights_best.csv')
#original_portfolio['Security\'s Percentage of the Total Net Assets'] = np.round(original_portfolio['Security\'s Percentage of the Total Net Assets'],3)/100
original_portfolio.index = original_portfolio['Ticker Symbol Given by the Exchange']
del original_portfolio['Ticker Symbol Given by the Exchange']
original_portfolio = original_portfolio.loc[top_30,]

In [11]:
# normalize top 30 weights
original_portfolio['Security\'s Percentage of the Total Net Assets'] = (original_portfolio['Security\'s Percentage of the Total Net Assets']/original_portfolio['Security\'s Percentage of the Total Net Assets'].sum())

In [12]:
# drop nans
simple_returns = simple_returns.dropna()

# get in-sample time window to perform optimization on (inclusive)
is_start_date = '2019-6-30'
is_end_date = '2020-6-30'

# get out-of-sample time window for evaluation (inclusive)
oos_start_date = '2020-7-1'
oos_end_date = '2020-12-31'

in_sample_window = simple_returns[(simple_returns.index>=is_start_date)&(simple_returns.index<=is_end_date)].copy()
out_sample_window = simple_returns[(simple_returns.index>=oos_start_date)&(simple_returns.index<=oos_end_date)].copy()

In [13]:
# create dataframe to store results
pd.DataFrame(np.zeros(len(in_sample_window.columns)+1),index = np.append(in_sample_window.columns.values,'RF'))

comparison_table = pd.DataFrame(original_portfolio['Security\'s Percentage of the Total Net Assets'])
comparison_table = comparison_table.rename(columns={'Security\'s Percentage of the Total Net Assets':'original weights'})
comparison_table.loc['RF','original weights'] = 0

In [14]:
# Calculate expected returns and sample covariance using PyPortfolioOpt’s built-in methods
mu = expected_returns.mean_historical_return(in_sample_window, returns_data=True)
mu['RF'] = 0.01 # add risk-free assets
S = risk_models.sample_cov(in_sample_window, returns_data=True)
S.loc['RF',:] = 0 # add risk-free assets
S.loc[:,'RF'] = 0

oos_mu = expected_returns.mean_historical_return(out_sample_window, returns_data=True)
oos_mu.loc['RF'] = 0.01 # add risk-free assets
oos_S = risk_models.sample_cov(out_sample_window, returns_data=True)
oos_S.loc['RF',:] = 0 # add risk-free assets
oos_S.loc[:,'RF'] = 0

#print('Expected Returns:')
#print(mu)
#print('\n')
#print('Covariance Matrix:')
#print(S)
solver = "ECOS"

# Attempt 1 -- Simple implementation
* Simple sub-sectors, semiconductors and others
* Base-line

In [15]:
# Set each stock to a sub-sector 
# Until someone has a better idea, use:
semiconductors = ['ADI','AMAT','AMD','AVGO','CRUS',
                  'DIOD','INTC','LRCX','MCHP','MRVL',
                  'MTSI','MU','MXL','NVDA','NXPI',
                  'OLED','ON','QCOM','SGH','SMTC',
                  'SYNA','TSM','TXN','XLNX']
others = ['AEIS','ENPH','FLEX','JBL','TTMI', 
               'IIVI','RF'] # AEIS and IIVI are also in the semiconductor subsector, but just putting them here first since they're in both areas. Every stock here except for IIVI are in the electrical/electronics scene. RF = risk-free

**Calculating risk aversion**

$U=E(R)-k*std(R)^2$

Assuming a portfolio consisting only of the market portfolio + risk-free assets, and expressing $E(R)$ and $std(R)$ in terms of their weights, $w$, we have:

$U=0.1*w+0.01*(1-w) - k*(w*0.18)^2$

where $w$ is the weight allocated to the market portfolio. Then,

$dU/dw = -0.0648*w*k+0.099$

If we choose w = 1, and then solving for $dU/dw = 0$, we get $k=1.528$

In [16]:
ef = EfficientFrontier(mu, S, solver= solver)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in semiconductors:
  sector_mapper[stock] = 'semiconductors'
for stock in others:
  sector_mapper[stock] = 'others'

sector_lower = {'others': 0.3} # at least 30% to others

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper={}) # add sector constraint

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
print(ef.portfolio_performance(verbose=True, risk_free_rate=0.01))
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = np.append(in_sample_window.columns, 'RF'))
print(results)
# check against out of sample window
_ = base_optimizer.portfolio_performance(results, oos_mu, oos_S, verbose=True, risk_free_rate=0.01)

k = 1.528
Expected annual return: 125.6%
Annual volatility: 53.6%
Sharpe Ratio: 2.32
(1.2555892515033282, 0.5360763269173905, 2.3235296709814866)
ADI     0.00
AEIS    0.00
AMAT    0.00
AMD     0.00
AVGO    0.00
CRUS    0.00
DIOD    0.00
ENPH    0.25
FLEX    0.00
IIVI    0.00
INTC    0.00
JBL     0.00
LRCX    0.00
MCHP    0.00
MRVL    0.00
MTSI    0.25
MU      0.00
MXL     0.00
NVDA    0.25
NXPI    0.00
OLED    0.00
ON      0.00
QCOM    0.00
SGH     0.00
SMTC    0.00
SYNA    0.20
TSM     0.00
TTMI    0.00
TXN     0.00
XLNX    0.00
RF      0.05
dtype: float64
Expected annual return: 392.3%
Annual volatility: 37.8%
Sharpe Ratio: 10.35


In [17]:
comparison_table['1'] = results.copy()

# Attempt 2 -- Addressing the 0 weights problem the mean-variance optimization function causes
* Simple sub-sectors, semiconductors and others
* L2 regularization with gamma = 5

**From the pyportfolio webpage**

*In order to coerce the mean-variance optimizer to produce more non-negligible weights, we add what can be thought of as a “small weights penalty” to all of the objective functions, parameterised by γ (gamma).*

It is refered to as L2 regularisation despite having a different purpose from its usual use, to keep weights small. Here, is it used to keep weights bigger.

Note: Check different gamma values and portfolio's overall performance.

https://pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html#pypfopt.objective_functions.L2_reg

In [18]:
# Set each stock to a sub-sector 
# Until someone has a better idea, use:
semiconductors = ['ADI','AMAT','AMD','AVGO','CRUS',
                  'DIOD','INTC','LRCX','MCHP','MRVL',
                  'MTSI','MU','MXL','NVDA','NXPI',
                  'OLED','ON','QCOM','SGH','SMTC',
                  'SYNA','TSM','TXN','XLNX']
others = ['AEIS','ENPH','FLEX','JBL','TTMI', 
               'IIVI','RF'] # AEIS and IIVI are also in the semiconductor subsector, but just putting them here first since they're in both areas. Every stock here except for IIVI are in the electrical/electronics scene. RF = risk-free

In [19]:
ef = EfficientFrontier(mu, S, solver= solver)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in semiconductors:
  sector_mapper[stock] = 'semiconductors'
for stock in others:
  sector_mapper[stock] = 'others'

sector_lower = {'others': 0.3} # at least 30% to others

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper={}) # add sector constraint

ef.add_objective(objective_functions.L2_reg, gamma=5) # L2 regularisation parameter, defaults to 1. This adds a small weights penalty, gamma, to the objective function. Increase for more non-negligible weights. 

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
print(ef.portfolio_performance(verbose=True, risk_free_rate=0.01))
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = np.append(in_sample_window.columns, 'RF'))
print(results)
# check against out of sample window
_ = base_optimizer.portfolio_performance(results, oos_mu, oos_S, verbose=True, risk_free_rate=0.01)

k = 1.528
Expected annual return: 79.8%
Annual volatility: 45.4%
Sharpe Ratio: 1.74
(0.7982307492835339, 0.45427751747745115, 1.7351304411023578)
ADI     0.0021
AEIS    0.0155
AMAT    0.0166
AMD     0.0648
AVGO    0.0066
CRUS    0.0255
DIOD    0.0328
ENPH    0.1256
FLEX    0.0080
IIVI    0.0394
INTC    0.0246
JBL     0.0111
LRCX    0.0509
MCHP    0.0027
MRVL    0.0415
MTSI    0.1082
MU      0.0163
MXL     0.0000
NVDA    0.1170
NXPI    0.0025
OLED    0.0000
ON      0.0000
QCOM    0.0202
SGH     0.0000
SMTC    0.0000
SYNA    0.0966
TSM     0.0572
TTMI    0.0289
TXN     0.0140
XLNX    0.0000
RF      0.0715
dtype: float64
Expected annual return: 255.3%
Annual volatility: 29.7%
Sharpe Ratio: 8.57


In [20]:
comparison_table['2'] = results.copy()

# Attempt 3 -- Addressing small number of sub-sectors issue
- Sub-sectors by market cap, with floor = 20%, and ceiling = 50%
- using L2 regularization with gamma = 5

***Additionally, we can consider putting a stock of a company into multiple sub-sectors, if the company produces goods of a different sub-sector as well. This has yet to be implemented, though I might get to it if I have time.
If someone wants to pick up the mantle and do the research, please go ahead.***

In [21]:
# market cap by three categories, 100B+, 10B+, and 1B+
cat_1 = ["TSM", "NVDA", "INTC", "AVGO", "TXN", "QCOM", "AMD", "AMAT"] # 100B+ market cap
cat_2 = ["ADI", "MU", "LRCX", "MRVL", "NXPI", "MCHP", "XLNX", "ENPH", "ON"] # 10B+ market cap
cat_3 = ["JBL", "FLEX", "OLED", "SYNA", "IIVI", "SMTC", "CRUS", "MTSI", "DIOD", "MXL", "AEIS", "TTMI", "SGH"] # 1B+ market cap

In [22]:
ef = EfficientFrontier(mu, S, solver= solver)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in cat_1:
  sector_mapper[stock] = 'cat_1'
for stock in cat_2:
  sector_mapper[stock] = 'cat_2'
for stock in cat_3:
  sector_mapper[stock] = 'cat_3'
sector_mapper['RF'] = 'rf'

sector_lower = {'cat_1': 0.2, 'cat_2': 0.2, 'cat_3': 0.2} # at least 30% to each category
sector_upper = {'cat_1': 0.5, 'cat_2': 0.5, 'cat_3': 0.5} # not more than 40% to each category

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper) # add sector constraint

ef.add_objective(objective_functions.L2_reg, gamma=5) # L2 regularisation parameter, defaults to 1. This adds a small weights penalty, gamma, to the objective function. Increase for more non-negligible weights. 

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
ef.portfolio_performance(verbose=True, risk_free_rate=0.01) 
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = np.append(in_sample_window.columns, 'RF'))
print(results)
print('\n')
print(f'CAT 1: {results.loc[cat_1].sum()}, CAT 2: {results.loc[cat_2].sum()}, CAT 3: {results.loc[cat_3].sum()}')
# check against out of sample window
_ = base_optimizer.portfolio_performance(results, oos_mu, oos_S, verbose=True, risk_free_rate=0.01)

k = 1.528
Expected annual return: 81.1%
Annual volatility: 45.6%
Sharpe Ratio: 1.76
ADI     0.0068
AEIS    0.0017
AMAT    0.0211
AMD     0.0689
AVGO    0.0113
CRUS    0.0302
DIOD    0.0377
ENPH    0.1144
FLEX    0.0000
IIVI    0.0259
INTC    0.0289
JBL     0.0000
LRCX    0.0554
MCHP    0.0072
MRVL    0.0461
MTSI    0.1130
MU      0.0207
MXL     0.0000
NVDA    0.1213
NXPI    0.0072
OLED    0.0000
ON      0.0000
QCOM    0.0248
SGH     0.0028
SMTC    0.0000
SYNA    0.1012
TSM     0.0619
TTMI    0.0154
TXN     0.0185
XLNX    0.0000
RF      0.0576
dtype: float64


CAT 1: 0.3567, CAT 2: 0.25780000000000003, CAT 3: 0.3279
Expected annual return: 244.6%
Annual volatility: 29.7%
Sharpe Ratio: 8.19


In [23]:
comparison_table['3'] = results.copy()

# Attempt 4 - Adjusting the mean and variance to be more realistic
- L2 regularization with gamma = 10 (5 had too many zero weights, only -0.05 to out of sample Sharpe Ratio)
- Exponentially weighted mean
- Shrinkage Covariance (trial of different shrinkage algorithms)
- Sub-sectors by market cap


***Personal thoughts on shrinkage: TA had a really excited reaction when I mentioned about shrinkage estimators, then talked about 
other methods we have yet to try. We don't really have time to read through the paper about shrinkage to find out the best method,
so I tried everything available in the pypfopt package. They all give extremely similar results, so I think we can proceed with the 
default implementation of it. I believe there are other shrinkage methods out there not implemented with this package, but unless
someone wants to go the extra mile to look up which is the best for our use, this is where I'll stop with shrinkages.***

***We can provide a table of the results of the different shrinkage implementations, but I believe we'll also need to justify
our final choice of shrinkage. I doubt they'll accept a quantitative explanation, but we can ask about this on Thursday.***

In [24]:
adj_mu = expected_returns.ema_historical_return(in_sample_window, span=250, returns_data=True)
adj_mu['RF'] = 0.01 # add risk-free assets
#adj_S = risk_models.exp_cov(in_sample_window, span=100, returns_data=True)

# Shrinkage estimator, as recommended by PyPortfolioOpt's dev
adj_S = risk_models.CovarianceShrinkage(in_sample_window, returns_data=True).ledoit_wolf('constant_variance') # 8.71 sharpe, default method offered 
#adj_S = risk_models.CovarianceShrinkage(in_sample_window, returns_data=True).ledoit_wolf('single_factor') # 8.68 sharpe
#adj_S = risk_models.CovarianceShrinkage(in_sample_window, returns_data=True).ledoit_wolf('constant_correlation') # 8.55 sharpe
#adj_S = risk_models.CovarianceShrinkage(in_sample_window, returns_data=True).oracle_approximating() # 8.70 sharpe
#adj_S = risk_models.CovarianceShrinkage(in_sample_window, returns_data=True).shrunk_covariance() # 8.76 sharpe (there is an optimal shrinkage constant that is normally estimated, but this function requires manual input for it)

adj_S.loc['RF',:] = 0 # add risk-free assets
adj_S.loc[:,'RF'] = 0

In [25]:
ef = EfficientFrontier(adj_mu, adj_S, solver= solver)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in cat_1:
  sector_mapper[stock] = 'cat_1'
for stock in cat_2:
  sector_mapper[stock] = 'cat_2'
for stock in cat_3:
  sector_mapper[stock] = 'cat_3'
sector_mapper['RF'] = 'rf'

sector_lower = {'cat_1': 0.2, 'cat_2': 0.2, 'cat_3': 0.2} # at least 20% to each category
sector_upper = {'cat_1': 0.5, 'cat_2': 0.5, 'cat_3': 0.5} # not more than 50% to each category

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper) # add sector constraint

ef.add_objective(objective_functions.L2_reg, gamma=10) # L2 regularisation parameter, defaults to 1. This adds a small weights penalty, gamma, to the objective function. Increase for more non-negligible weights. 

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
#ef.portfolio_performance(verbose=True, risk_free_rate=0.01) # can't use this anymore, its adjusted mean
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = np.append(in_sample_window.columns, 'RF'))
_ = base_optimizer.portfolio_performance(results, mu, S, verbose=True, risk_free_rate=0.01)
print(results)
print('\n')
print(f'CAT 1: {results.loc[cat_1].sum()}, CAT 2: {results.loc[cat_2].sum()}, CAT 3: {results.loc[cat_3].sum()}')
# check against out of sample window
_ = base_optimizer.portfolio_performance(results, oos_mu, oos_S, verbose=True, risk_free_rate=0.01)

k = 1.528
Expected annual return: 68.8%
Annual volatility: 49.5%
Sharpe Ratio: 1.37
ADI     0.0203
AEIS    0.0262
AMAT    0.0239
AMD     0.0390
AVGO    0.0245
CRUS    0.0002
DIOD    0.0253
ENPH    0.1406
FLEX    0.0053
IIVI    0.0650
INTC    0.0165
JBL     0.0079
LRCX    0.0507
MCHP    0.0281
MRVL    0.0615
MTSI    0.0835
MU      0.0174
MXL     0.0365
NVDA    0.1051
NXPI    0.0182
OLED    0.0000
ON      0.0210
QCOM    0.0284
SGH     0.0088
SMTC    0.0205
SYNA    0.0293
TSM     0.0288
TTMI    0.0090
TXN     0.0170
XLNX    0.0150
RF      0.0263
dtype: float64


CAT 1: 0.2832, CAT 2: 0.3728, CAT 3: 0.31749999999999995
Expected annual return: 272.3%
Annual volatility: 31.2%
Sharpe Ratio: 8.71


In [26]:
comparison_table['4'] = results.copy()

# Attempt 5 -- Semivariance in place of variance
- L2 regularization with gamma = 5
- Exponentially moving mean, Semivariance in place of variance
- Sub-sectors by market cap

The semivariance is the variance of all returns which are below some benchmark. In this case, the benchmark is the risk-free rate. Although using the semi-covariance in terms of covariance in mean-variance optimization has been shown to not yield a portfolio efficient in semicovariance space, it can function as a heuristical approximation.

In [27]:
adj_mu = expected_returns.ema_historical_return(in_sample_window, span=250, returns_data=True)
adj_mu['RF'] = 0.01 # add risk-free assets

# Semicovariance
adj_S = risk_models.semicovariance(in_sample_window, returns_data=True, benchmark=(1.01)**(1/252)-1)

adj_S.loc['RF',:] = 0 # add risk-free assets
adj_S.loc[:,'RF'] = 0

In [28]:
ef = EfficientFrontier(adj_mu, adj_S, solver= solver)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in cat_1:
  sector_mapper[stock] = 'cat_1'
for stock in cat_2:
  sector_mapper[stock] = 'cat_2'
for stock in cat_3:
  sector_mapper[stock] = 'cat_3'
sector_mapper['RF'] = 'rf'

sector_lower = {'cat_1': 0.2, 'cat_2': 0.2, 'cat_3': 0.2} # at least 20% to each category
sector_upper = {'cat_1': 0.5, 'cat_2': 0.5, 'cat_3': 0.5} # not more than 50% to each category

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper) # add sector constraint

ef.add_objective(objective_functions.L2_reg, gamma=10) # L2 regularisation parameter, defaults to 1. This adds a small weights penalty, gamma, to the objective function. Increase for more non-negligible weights. 

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
#ef.portfolio_performance(verbose=True, risk_free_rate=0.01) # can't use this anymore, its adjusted mean
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = np.append(in_sample_window.columns, 'RF'))
_ = base_optimizer.portfolio_performance(results, mu, S, verbose=True, risk_free_rate=0.01)
print(results)
print('\n')
print(f'CAT 1: {results.loc[cat_1].sum()}, CAT 2: {results.loc[cat_2].sum()}, CAT 3: {results.loc[cat_3].sum()}')
# check against out of sample window
_ = base_optimizer.portfolio_performance(results, oos_mu, oos_S, verbose=True, risk_free_rate=0.01)

k = 1.528
Expected annual return: 70.5%
Annual volatility: 50.8%
Sharpe Ratio: 1.37
ADI     0.0204
AEIS    0.0284
AMAT    0.0254
AMD     0.0373
AVGO    0.0228
CRUS    0.0009
DIOD    0.0241
ENPH    0.1510
FLEX    0.0059
IIVI    0.0647
INTC    0.0141
JBL     0.0060
LRCX    0.0543
MCHP    0.0308
MRVL    0.0610
MTSI    0.0849
MU      0.0189
MXL     0.0407
NVDA    0.1066
NXPI    0.0187
OLED    0.0000
ON      0.0253
QCOM    0.0263
SGH     0.0106
SMTC    0.0224
SYNA    0.0274
TSM     0.0241
TTMI    0.0073
TXN     0.0149
XLNX    0.0120
RF      0.0128
dtype: float64


CAT 1: 0.2715, CAT 2: 0.3924, CAT 3: 0.3233
Expected annual return: 285.3%
Annual volatility: 31.9%
Sharpe Ratio: 8.91


In [29]:
comparison_table['5'] = results.copy()

# Attempt 6 -- Black-Litterman allocation
- L2 regularization with gamma = 5
- Mean and Variance are estimated using black-litterman allocation
- Black-Litterman allocation's view is estimated with the EMA of the mean.
- Sub-sectors by market cap

Refer: https://pyportfolioopt.readthedocs.io/en/latest/BlackLitterman.html

Unless there's a large flaw in the logic of using EMA as the view, I expect we'll be using this method as our chosen method.


In [30]:
k = 1.528 
def calc_risk_aversion(market_returns, frequency = 252, risk_free_rate = 0.01):
    '''
    Function to obtain the the risk aversion of individual funds given the market returns. A modification of the market_implied_risk_aversion function from pypfopt's black_litterman implementation.
    This modification takes in the daily market returns instead of the daily market prices. The original function also returns values un-compounded. Here, they are compounded.
    
    Parameters:
    - market_returns (pd.DataFrame): Dataframe of the daily percentage returns. Tickers in the columns, and dates in the index. 
    - frequency (int, optional): number of time periods in a year, defaults to 252 (the number of trading days in a year)
    - risk_free_rate (float, optional): risk-free rate of borrowing/lending, defaults to 0.01. The period of the risk-free rate should correspond to the frequency of expected returns.
    
    Returns:
    - market-implied risk aversion (pd.Series)
    '''
    
    r = (1+market_returns.mean()) ** frequency - 1
    var = market_returns.var() * frequency
    return (r - risk_free_rate) / var

# read and process market cap data into a suitable format
market_caps = pd.read_csv('market_cap_data.csv')
market_caps2 = pd.Series(data=market_caps['MARKET CAP IN BILLIONS'].values)
market_caps2.index = market_caps['TICKER'].values
market_caps = market_caps2[market_caps2.index != 'RF']
market_caps = market_caps.sort_index()

# use the black-litterman allocation to generate the adjusted mean and variance

cov_matrix = risk_models.CovarianceShrinkage(in_sample_window, returns_data=True).ledoit_wolf('constant_variance') # using a shrinkage covariance estimator

risk_aversion = calc_risk_aversion(in_sample_window, frequency=252, risk_free_rate=0.01)
prior_returns = black_litterman.market_implied_prior_returns(market_caps, risk_aversion, cov_matrix, risk_free_rate=0.01) # priors can be thought of the default estimate, in absense of information. 
                                                                                                                          # A natural choice for this prior is the market’s estimate of the return, 
                                                                                                                          # which is embedded into the market capitalisation of the asset.


views = expected_returns.ema_historical_return(in_sample_window, span=250, returns_data=True) # here, we use the exponentially moving average mean historical return to represent our views on how the stock will grow

bl_model = black_litterman.BlackLittermanModel(cov_matrix, pi=prior_returns, absolute_views=views, risk_aversion=k) 

adj_mu = bl_model.bl_returns() # posterior mean returns
adj_mu['RF'] = 0.01 # add risk-free assets
adj_S = bl_model.bl_cov() # posterior covariance
adj_S.loc['RF',:] = 0 # add risk-free assets
adj_S.loc[:,'RF'] = 0

In [31]:
ef = EfficientFrontier(adj_mu, adj_S, solver=solver)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in cat_1:
  sector_mapper[stock] = 'cat_1'
for stock in cat_2:
  sector_mapper[stock] = 'cat_2'
for stock in cat_3:
  sector_mapper[stock] = 'cat_3'
sector_mapper['RF'] = 'rf'

sector_lower = {'cat_1': 0.2, 'cat_2': 0.2, 'cat_3': 0.2} # at least 20% to each category
sector_upper = {'cat_1': 0.5, 'cat_2': 0.5, 'cat_3': 0.5} # not more than 50% to each category

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper) # add sector constraint

ef.add_objective(objective_functions.L2_reg, gamma=5) # L2 regularisation parameter, defaults to 1. This adds a small weights penalty, gamma, to the objective function. Increase for more non-negligible weights. 

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = np.append(in_sample_window.columns, 'RF'))
_ = base_optimizer.portfolio_performance(results, mu, S, verbose=True, risk_free_rate=0.01)
print(results)
print('\n')
print(f'CAT 1: {results.loc[cat_1].sum()}, CAT 2: {results.loc[cat_2].sum()}, CAT 3: {results.loc[cat_3].sum()}')
# check against out of sample window
_ = base_optimizer.portfolio_performance(results, oos_mu, oos_S, verbose=True, risk_free_rate=0.01)

k = 1.528
Expected annual return: 76.2%
Annual volatility: 48.1%
Sharpe Ratio: 1.56
ADI     0.0174
AEIS    0.0184
AMAT    0.0290
AMD     0.0567
AVGO    0.0161
CRUS    0.0147
DIOD    0.0293
ENPH    0.1230
FLEX    0.0001
IIVI    0.0498
INTC    0.0246
JBL     0.0049
LRCX    0.0558
MCHP    0.0232
MRVL    0.0568
MTSI    0.0961
MU      0.0260
MXL     0.0101
NVDA    0.1254
NXPI    0.0114
OLED    0.0000
ON      0.0040
QCOM    0.0266
SGH     0.0094
SMTC    0.0121
SYNA    0.0521
TSM     0.0553
TTMI    0.0108
TXN     0.0167
XLNX    0.0000
RF      0.0241
dtype: float64


CAT 1: 0.35040000000000004, CAT 2: 0.3176, CAT 3: 0.30779999999999996
Expected annual return: 254.4%
Annual volatility: 30.7%
Sharpe Ratio: 8.24


In [32]:
comparison_table['6'] = results.copy()

## 3. Evaluation

Add any other relevant data.
Even if we don't need to compare old vs new portfolio, we still need this data for VaR calculations

In [33]:
comparison_table

,original weights,1,2,3,4,5,6
Ticker Symbol Given by the Exchange,,,,,,,
SYNA,0.005068,0.20,0.0966,0.1012,0.0293,0.0274,0.0521
MRVL,0.055538,0.00,0.0415,0.0461,0.0615,0.0610,0.0568
SGH,0.004757,0.00,0.0000,0.0028,0.0088,0.0106,0.0094
ENPH,0.029372,0.25,0.1256,0.1144,0.1406,0.1510,0.1230
JBL,0.014479,0.00,0.0111,0.0000,0.0079,0.0060,0.0049
CRUS,0.021512,0.00,0.0255,0.0302,0.0002,0.0009,0.0147
MCHP,0.049436,0.00,0.0027,0.0072,0.0281,0.0308,0.0232
AEIS,0.007550,0.00,0.0155,0.0017,0.0262,0.0284,0.0184
MTSI,0.007033,0.25,0.1082,0.1130,0.0835,0.0849,0.0961


# Q4 VaR and ES


In [34]:
def get_historical_risk(returns_df, rolling_window_size, dates, weights, risk_free_rate = 0.01):
    '''
    Function to obtain the historical risk given the dates provided. Assumes 5% VaR and 5% ES.
    
    Parameters:
    - returns_df (pd.DataFrame): Dataframe of the daily percentage returns. Tickers in the columns, and dates in the index. 
    - rolling_window_size (int): Integer ditating the size of the rolling window to consider when calculating the historical risk.
    - dates (list-like): Integer or list-like object containing the dates to calculate historical VaR for.
    - weights (pd.Series): Pandas series object containing the weights of each investment. Index is the ticker names for each investment.
    
    Returns:
    - risk_df (pd.DataFrame): Dataframe of the log VaR and ES of each provided date. Dates are the index.
    '''
    
    if isinstance(dates, dict):
        if ('start_date' or 'end_date') not in dates.keys():
            raise ValueError("Missing start_date or end_date variable.")
        start_date = dates['start_date']
        end_date = dates['end_date']
        dates = returns_df[(returns_df.index>=start_date)&(returns_df.index<=end_date)].index.values
    
    elif not isinstance(dates, list):
        dates = [dates,]
    
    not_found = [date for date in dates if date not in returns_df.index]
    if any(not_found):
        raise ValueError(f"{not_found} not found in provided data")
        
    returns_df.loc[:,'RF'] = (1+risk_free_rate)**(1/252) -1
        
    historical_returns = np.log((returns_df*weights).sum(axis=1)+1) # calculate the historical returns of the portfolio after weighting, then log it
    historical_var = np.zeros(len(dates)) # create empty array for fast data read/write
    historical_ES = np.zeros(len(dates)) # create empty array for fast data read/write
    
    for day_index in range(len(dates)): # for each date
        date = dates[day_index]
        rolling_window = historical_returns[historical_returns.index<date].iloc[-rolling_window_size:]
        historical_var[day_index] = np.percentile(rolling_window,5) # get 5% percentile for VaR
        sum_of_loss_returns = np.sum(rolling_window.iloc[np.where(rolling_window < historical_var[day_index])]) # formula for ES, select all values lower than the 5% VaR and sum them.
        historical_ES[day_index] = sum_of_loss_returns/(rolling_window_size*0.05) # formula for ES
    
    risk_df = pd.DataFrame([historical_var,historical_ES],index=['5% VaR', '5% ES'],columns=dates).T
    
    return risk_df
        

In [35]:
# NOTE: LOG
from scipy.stats import norm
def get_parametric_risk(returns_df, rolling_window_size, dates, weights, risk_free_rate = 0.01):
    '''
    Function to obtain the parametric risk given the dates provided. Assumes 5% VaR and 5% ES, and normal distribution.
    
    Parameters:
    - returns_df (pd.DataFrame): Dataframe of the daily percentage returns. Tickers in the columns, and dates in the index. 
    - rolling_window_size (int): Integer ditating the size of the rolling window to consider when calculating the historical risk.
    - dates (list-like): Integer or list-like object containing the dates to calculate historical VaR for.
    - weights (pd.Series): Pandas series object containing the weights of each investment. Index is the ticker names for each investment.
    
    Returns:
    - risk_df (pd.DataFrame): Dataframe of the log VaR and ES of each provided date. Dates are the index.
    '''
    
    if isinstance(dates, dict):
        if ('start_date' or 'end_date') not in dates.keys():
            raise ValueError("Missing start_date or end_date variable.")
        start_date = dates['start_date']
        end_date = dates['end_date']
        dates = returns_df[(returns_df.index>=start_date)&(returns_df.index<=end_date)].index.values
    
    elif not isinstance(dates, list):
        dates = [dates,]
    
    not_found = [date for date in dates if date not in returns_df.index]
    if any(not_found):
        raise ValueError(f"{not_found} not found in provided data")
        
    returns_df.loc[:,'RF'] = (1+risk_free_rate)**(1/252) -1
        
    parametric_var = np.zeros(len(dates)) # create empty array for fast data read/write
    parametric_ES = np.zeros(len(dates)) # create empty array for fast data read/write
    
    for day_index in range(len(dates)): # for each date
        date = dates[day_index]
        
        individual_window = returns_df[returns_df.index<date].iloc[-rolling_window_size:].values # collect the values for the logreturns for each individual stock, inclusive of the day itself
        
        # calculate mean
        mu = expected_returns.mean_historical_return(individual_window, returns_data=True) # this gives annual returns
        mu = (mu+1)**(1/252)-1 # convert to daily results
        log_mean_return = np.log((mu.values*weights).sum() +1) # in log
        
        # calculate variance 
        S = np.log(risk_models.sample_cov(individual_window, returns_data=True)/252+1) # get covariance matrix, divide by 252 to get daily values, then log it
        
        variance_return = np.dot(weights, np.dot(S, weights)) # calculate portfolio variance

        # calculate VaR and ES
        volatility = variance_return**0.5
        z_alpha = norm.ppf(0.05)
        parametric_var[day_index] = log_mean_return + volatility*z_alpha
        parametric_ES[day_index] = log_mean_return-volatility*(norm.pdf(norm.ppf(0.05))/0.05)
    
    risk_df = pd.DataFrame([parametric_var,parametric_ES],index=['5% VaR', '5% ES'],columns=dates).T
    
    return risk_df

### Risk for new portfolio

In [36]:
historical_risk_values = get_historical_risk(returns_df=simple_returns, rolling_window_size=250, dates={"start_date":"2019-1-1","end_date":"2019-12-31"}, weights=results)
historical_risk_values

,5% VaR,5% ES
2019-01-02,-0.033265,-0.046266
2019-01-03,-0.033265,-0.046266
2019-01-04,-0.034067,-0.048128
2019-01-07,-0.034067,-0.048128
2019-01-08,-0.034067,-0.048128
...,...,...
2019-12-24,-0.025013,-0.038184
2019-12-26,-0.025013,-0.038184
2019-12-27,-0.025013,-0.038184
2019-12-30,-0.025013,-0.038184


In [37]:
parametric_risk_values = get_parametric_risk(returns_df=simple_returns, rolling_window_size=250, dates={"start_date":"2019-1-1","end_date":"2019-12-31"}, weights=results)
parametric_risk_values

D:\Anaconda\lib\site-packages\pypfopt\expected_returns.py:119: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)
D:\Anaconda\lib\site-packages\pypfopt\risk_models.py:165: RuntimeWarning: data is not in a dataframe
  warnings.warn("data is not in a dataframe", RuntimeWarning)


,5% VaR,5% ES
2019-01-02,-0.032796,-0.040973
2019-01-03,-0.032787,-0.040957
2019-01-04,-0.033526,-0.041825
2019-01-07,-0.033748,-0.042143
2019-01-08,-0.033795,-0.042222
...,...,...
2019-12-24,-0.025223,-0.032316
2019-12-26,-0.025214,-0.032307
2019-12-27,-0.025213,-0.032306
2019-12-30,-0.025264,-0.032362


### Risk for old portfolio

In [38]:
# load stocks into a dictionary
bad_fund = {}
do_not_append = [] # FSELX is our fund, while IPHI does not have data and has been bought over by MRVL
for file_name in os.listdir('worst_data/'):
  ticker = file_name.split(".")[0]
  if ticker in do_not_append:
    pass
  else:
    bad_fund[f'{ticker}'] = pd.read_csv(f'worst_data/{file_name}') # for each stock, add its ticker as the key and dataframe as the value

In [39]:
print(list(bad_fund.keys())) # print the stock tickers for easy referencing

['AAPL', 'ABBV', 'ADBE', 'ADP', 'ADSK', 'AMZN', 'ANTM', 'BAH', 'EBIX', 'EEFT', 'ENS', 'EPAM', 'EXTR', 'G', 'GOOG', 'GPN', 'IIVI', 'IQV', 'JBL', 'MA', 'MSFT', 'NXPI', 'PRFT', 'QRVO', 'ROG', 'ST', 'SWKS', 'TEL', 'THR', 'UNH', 'V']


In [40]:
# Change date column into a datetime object
# Add a new column for log returns
for ticker, df in bad_fund.items():
  df['Date'] = df['Date'].apply(pd.to_datetime)
  df['LogReturns'] = np.append(np.nan,np.diff(np.log(df['Adj Close']))) # Fill dataframe with the log returns. The first value will be nan because there is no log returns for it.

In [41]:
# preview a stock
bad_fund['AAPL'].head()

,Date,Open,High,Low,Close,Adj Close,Volume,LogReturns
0,2016-10-04,28.264999,28.577499,28.157499,28.250000,26.493998,118947200,NaN
1,2016-10-05,28.350000,28.415001,28.172501,28.262501,26.505718,85812400,0.000442
2,2016-10-06,28.424999,28.584999,28.282499,28.472500,26.702660,115117200,0.007403
3,2016-10-07,28.577499,28.639999,28.377501,28.514999,26.742519,97433600,0.001492
4,2016-10-10,28.754999,29.187500,28.680000,29.012501,27.209097,144944000,0.017297


In [42]:
bad_fund_log_returns = pd.DataFrame(data=np.full((bad_fund['AAPL'].shape[0],31),np.nan), columns = sorted(list(bad_fund.keys())), index = bad_fund['AAPL'].Date) # create empty dataframe filled with NaNs, with index = Date and columns = stocks

# copy the log returns over from the stocks dictionary
for ticker, df in bad_fund.items():
  bad_fund_log_returns[ticker] = bad_fund_log_returns.index.map(bad_fund[ticker].set_index('Date')['LogReturns'])

# if simple returns are needed
bad_fund_simple_returns = np.exp(bad_fund_log_returns)-1
# set bad fund earliest date to the same as the good fund
bad_fund_simple_returns = bad_fund_simple_returns[bad_fund_simple_returns.index>=simple_returns.index[0]]

In [43]:
# preview log_returns
bad_fund_simple_returns.head()

,AAPL,ABBV,ADBE,ADP,ADSK,AMZN,ANTM,BAH,EBIX,EEFT,...,NXPI,PRFT,QRVO,ROG,ST,SWKS,TEL,THR,UNH,V
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-25,0.003457,0.005915,0.012259,0.011803,0.012536,0.013291,0.007885,0.016338,0.015302,-0.003303,...,-0.000463,0.038348,0.005613,0.017926,0.003912,0.005881,0.017176,-0.045842,0.014010,0.002426
2017-05-26,-0.001690,-0.003920,-0.006720,0.001878,-0.007551,0.002416,0.008705,0.001276,0.003107,-0.000473,...,0.000463,-0.041477,-0.012559,0.002213,-0.013639,-0.001414,-0.002047,-0.003352,-0.003089,-0.003893
2017-05-30,0.000391,-0.000606,0.003665,0.001776,0.001150,0.000924,0.001311,0.000255,-0.033629,0.004974,...,0.001296,0.019561,0.010406,-0.010120,0.003951,0.004439,0.012819,0.021300,-0.005127,0.000528
2017-05-31,-0.005922,0.000000,-0.003862,0.008273,-0.012284,-0.002087,-0.005291,0.004841,0.013736,0.027928,...,0.016275,0.004070,-0.008900,-0.012918,-0.005657,0.000658,-0.002025,0.016465,-0.007985,0.005384
2017-06-01,0.002749,0.010452,-0.003384,0.009573,0.011273,0.001337,0.029285,-0.000761,0.011743,0.016393,...,-0.005278,0.017950,0.014240,0.025704,0.016572,0.011745,0.012302,0.036717,0.025117,0.001785


In [44]:
# TO-DO: get original weights for the bad fund and calculate VaR